# Leitura e Formatação dos Dados de Desmatamento

In [2]:
import geopandas as gpd
import ast
import requests

# Carregar o arquivo GeoPackage
gdf = gpd.read_file(r"C:\Users\alanb\Documents\Desmatamento - Alertas [2024-11]\deforestation_alerts_hiwi.gpkg")
gdf.columns

Index(['is_valid', 'analysis_date', 'area', 'location', 'source_images_date',
       'source_images', 'internal_storage_path', 'geometry'],
      dtype='object')

In [39]:
# Função para baixar um arquivo
def download_file(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Download concluído: {save_path}")
    else:
        print(f"Falha no download: {url}")
        print(f"Nome: {save_path}")
        
def criar_urls_bandas(source_images, bands, resolutions):
    list_source_images = []
    for source_image in source_images:
        source_images_full = []
        if source_image['Band'] == 'B04':
            url = source_image['Url']
            name = source_image['Name']
            band = source_image['Band']
            for j in range(15):
                dict_link = url.replace('B04_10m', bands[j])
                dict_link = dict_link.replace('R10m', resolutions[j])
                
                dict_name = name.replace('B04_10m', bands[j])
                dict_band = band.replace('B04', bands[j].split('_')[0])
                
                dados = {'Name': dict_name, 'Band': dict_band, 'Url': dict_link}
                
                source_images_full.append(dados)
                
        list_source_images.append(source_images_full)
                
                
    return list_source_images    
            

In [37]:
list_bands_sentinel = ['B01_60m', 
                       'B02_10m',
                       'B03_10m',
                       'B04_10m',  
                       'B05_20m', 
                       'B06_20m',
                       'B07_20m',
                       'B08_10m',
                       'B8A_20m',
                       'B09_60m',
                       'B10_60m',
                       'B11_20m',
                       'B12_20m',
                       'SCL_20m',
                       'TCI_10m',]

list_bands_resolutions = ['R60m', 
                          'R10m', 
                          'R10m', 
                          'R10m', 
                          'R20m', 
                          'R20m', 
                          'R20m', 
                          'R10m',
                          'R20m',
                          'R60m',
                          'R60m',
                          'R20m',
                          'R20m',
                          'R20m',
                          'R10m']

In [43]:
import os
list_final_source_images = []
save_directory = 'C:/Users/alanb/Documents/'
#for i in range(1, 2):
row = gdf.iloc[20]
name_path = save_directory + row['internal_storage_path'] + '/'
source_images = ast.literal_eval(row['source_images'])
list_source_images = criar_urls_bandas(source_images, list_bands_sentinel, list_bands_resolutions)
#list_final_source_images.append(list_source_images)
for source_images in list_source_images:
    for source_image in source_images:
        url = source_image['Url']
        name = source_image['Name']
        os.makedirs(name_path, exist_ok=True)
        download_file(url, name_path + name)

Download concluído: C:/Users/alanb/Documents/PROD_feada872-2d2b-4f57-87c6-98fafe088254/20231028_502e7af2-b328-43cc-aaf4-f692218544f0/T19LEJ_20231028T144731_B01_60m.jp2
Download concluído: C:/Users/alanb/Documents/PROD_feada872-2d2b-4f57-87c6-98fafe088254/20231028_502e7af2-b328-43cc-aaf4-f692218544f0/T19LEJ_20231028T144731_B02_10m.jp2
Download concluído: C:/Users/alanb/Documents/PROD_feada872-2d2b-4f57-87c6-98fafe088254/20231028_502e7af2-b328-43cc-aaf4-f692218544f0/T19LEJ_20231028T144731_B03_10m.jp2
Download concluído: C:/Users/alanb/Documents/PROD_feada872-2d2b-4f57-87c6-98fafe088254/20231028_502e7af2-b328-43cc-aaf4-f692218544f0/T19LEJ_20231028T144731_B04_10m.jp2
Download concluído: C:/Users/alanb/Documents/PROD_feada872-2d2b-4f57-87c6-98fafe088254/20231028_502e7af2-b328-43cc-aaf4-f692218544f0/T19LEJ_20231028T144731_B05_20m.jp2
Download concluído: C:/Users/alanb/Documents/PROD_feada872-2d2b-4f57-87c6-98fafe088254/20231028_502e7af2-b328-43cc-aaf4-f692218544f0/T19LEJ_20231028T144731_B06_

In [32]:
list_final_source_images[0][0]

[{'Name': 'T19LEJ_20231102T144729_B01_60m.jp2',
  'Band': 'B01',
  'Url': 'https://storage.googleapis.com/gcp-public-data-sentinel-2/L2/tiles/19/L/EJ/S2B_MSIL2A_20231102T144729_N0509_R139_T19LEJ_20231102T185033.SAFE/GRANULE/L2A_T19LEJ_A034771_20231102T145104/IMG_DATA/R60m/T19LEJ_20231102T144729_B01_60m.jp2'},
 {'Name': 'T19LEJ_20231102T144729_B02_10m.jp2',
  'Band': 'B02',
  'Url': 'https://storage.googleapis.com/gcp-public-data-sentinel-2/L2/tiles/19/L/EJ/S2B_MSIL2A_20231102T144729_N0509_R139_T19LEJ_20231102T185033.SAFE/GRANULE/L2A_T19LEJ_A034771_20231102T145104/IMG_DATA/R10m/T19LEJ_20231102T144729_B02_10m.jp2'},
 {'Name': 'T19LEJ_20231102T144729_B03_10m.jp2',
  'Band': 'B03',
  'Url': 'https://storage.googleapis.com/gcp-public-data-sentinel-2/L2/tiles/19/L/EJ/S2B_MSIL2A_20231102T144729_N0509_R139_T19LEJ_20231102T185033.SAFE/GRANULE/L2A_T19LEJ_A034771_20231102T145104/IMG_DATA/R10m/T19LEJ_20231102T144729_B03_10m.jp2'},
 {'Name': 'T19LEJ_20231102T144729_B04_10m.jp2',
  'Band': 'B04',
  

In [25]:
len(gdf)

3947

In [10]:
row = gdf.iloc[0]
source_images = row['source_images']
source_images = ast.literal_eval(source_images)

# Diretório onde os arquivos serão salvos
save_directory = 'C:/Users/alanb/Documents'

In [ ]:
# Baixar cada arquivo da lista de links
for link in source_images:
    print(link['Url'])
    file_name = link['Url'].split('/')[-1]
    save_path = f"{save_directory}/{file_name}"
    download_file(link['Url'], save_path)

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon

# Supondo que row['geometry'] seja um objeto MultiPolygon
multipolygon = row['geometry']

# Criar uma figura e um eixo
fig, ax = plt.subplots()

# Plotar cada polígono no MultiPolygon
for polygon in multipolygon.geoms:
    x, y = polygon.exterior.xy
    ax.plot(x, y)

# Configurar os eixos
ax.set_aspect('equal')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MultiPolygon Plot')

# Mostrar o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import rasterio
from shapely.geometry import MultiPolygon
from shapely.ops import transform
from pyproj import Proj, Transformer

# Supondo que row['geometry'] seja um objeto MultiPolygon
multipolygon = row['geometry']

# Caminho para a imagem
image_path = r"C:\Users\alanb\Documents\TCI_merged.tiff"

# Ler a imagem usando rasterio
with rasterio.open(image_path) as src:
    image = src.read([1, 2, 3])  # Ler as três primeiras bandas (RGB)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
    image_crs = src.crs  # Sistema de coordenadas da imagem

# Sistema de coordenadas dos polígonos (supondo que seja WGS84)
polygons_crs = 'EPSG:4326'

# Criar um transformador para converter as coordenadas
transformer = Transformer.from_crs(polygons_crs, image_crs, always_xy=True)

# Função para transformar as coordenadas dos polígonos
def transform_polygon(polygon):
    return transform(transformer.transform, polygon)

# Transformar os polígonos para o sistema de coordenadas da imagem
transformed_multipolygon = MultiPolygon([transform_polygon(polygon) for polygon in multipolygon.geoms])

# Criar uma figura e um eixo
fig, ax = plt.subplots(figsize=(10, 10))

# Plotar a imagem de fundo
ax.imshow(image.transpose(1, 2, 0), extent=extent)

# Plotar cada polígono no MultiPolygon transformado
for polygon in transformed_multipolygon.geoms:
    x, y = polygon.exterior.xy
    ax.plot(x, y, color='red')
    print(x, y)

# Configurar os eixos
ax.set_aspect('equal')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MultiPolygon Plot with Background Image')

# Mostrar o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import rasterio
from shapely.geometry import MultiPolygon
from shapely.ops import transform
from pyproj import Proj, Transformer
import numpy as np


# Supondo que row['geometry'] seja um objeto MultiPolygon
multipolygon = row['geometry']

# Caminho para a imagem
image_path = r"C:\Users\alanb\Documents\B04_merged.tiff"

# Ler a imagem usando rasterio
with rasterio.open(image_path) as src:
    image = src.read()  # Ler as três primeiras bandas (RGB)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
    image_crs = src.crs  # Sistema de coordenadas da imagem

# Sistema de coordenadas dos polígonos (supondo que seja WGS84)
polygons_crs = 'EPSG:4326'

# Criar um transformador para converter as coordenadas
transformer = Transformer.from_crs(polygons_crs, image_crs, always_xy=True)

# Função para transformar as coordenadas dos polígonos
def transform_polygon(polygon):
    return transform(transformer.transform, polygon)

# Transformar os polígonos para o sistema de coordenadas da imagem
transformed_multipolygon = MultiPolygon([transform_polygon(polygon) for polygon in multipolygon.geoms])

# Criar uma figura e um eixo
fig, ax = plt.subplots(figsize=(10, 10))

# Plotar a imagem de fundo
ax.imshow(np.squeeze(image), extent=extent)

# Plotar cada polígono no MultiPolygon transformado
for polygon in transformed_multipolygon.geoms:
    x, y = polygon.exterior.xy
    ax.plot(x, y, color='red')
    print(x, y)

# Configurar os eixos
ax.set_aspect('equal')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('MultiPolygon Plot with Background Image')

# Mostrar o gráfico
plt.show()

In [ ]:
image.shape